# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [2]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2025-10-16 11:44:24--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M  --.-KB/s    in 0.1s    

2025-10-16 11:44:26 (330 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [3]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [4]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [5]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

In [20]:
# 1. Mapeamento: conta palavras em cada idioma
def map2(row):
    qtde_en = len(row.text_en.split())
    qtde_pt = len(row.text_pt.split())
    return (row.sentiment, (qtde_en, qtde_pt))

# 2. Redução: soma os totais de en e pt separadamente
def reduceByKey2(a, b):
    return (a[0] + b[0], a[1] + b[1])

# 3. Filtra apenas ‘neg’ e executa MapReduce
neg_totals = (
    imdb_df
      .filter("sentiment = 'neg'")
      .rdd
      .map(map2)
      .reduceByKey(reduceByKey2)
      .collect()
)

# 4. Calcula a diferença en – pt fora do reduce
sent, (total_en, total_pt) = neg_totals[0]
diferenca_neg = total_en - total_pt

print(f"diferença de palavras (en – pt) em 'neg': {diferenca_neg}")
print("RU: 4871164")


diferença de palavras (en – pt) em 'neg': -54949
RU: 4871164


In [19]:
from pyspark.sql.functions import split, size, sum as _sum

# 1. Filtra apenas resenhas negativas
df_neg = imdb_df.filter("sentiment = 'neg'")

# 2. Somas de palavras por idioma
soma_pt = _sum(size(split("text_pt", " ")))
soma_en = _sum(size(split("text_en", " ")))

# 3. Calcula a diferença pt – en
df_diff = df_neg.select((soma_pt - soma_en).alias("diff_pt_en"))

# 4. Exibe o resultado
df_diff.show()


+----------+
|diff_pt_en|
+----------+
|     54036|
+----------+

